[View in Colaboratory](https://colab.research.google.com/github/zbkruturaj/python_blog_resources/blob/master/notebooks/TicTacToe_pipeline.ipynb)

In [1]:
from collections import OrderedDict, defaultdict
import itertools
from random import choice
from time import sleep
from google.colab import widgets

def fix_google_colab_widgets():
  """
  The clear_cell function in google.colab.widgets._grid is buggy. A patch has 
  been sent to their github repo. This is a temperory patch till that patch is 
  made live.
  """
  
  def new_clear_cell(self, row=None, col=None):
      """Clears given cell. If row/col are None clears active cell."""
      if row is not None:
        if row < 0 or row >= self.rows:
          raise ValueError('%d is not a valid row' % row)
        if col < 0 or col >= self.columns:
          raise ValueError('%d is not a valid column' % col)
        cellid = self._get_cell_id(row, col)
      else:
        cellid = None
      self._clear_component(cellid)
  widgets.Grid.clear_cell = new_clear_cell

class FixedDict(object):
    """A dictionary class with ordered default dictionary and immutable keys."""
    
    def __init__(self, keys, default):
      self._dictionary = OrderedDict({k: default() for k in keys})
    
    def __setitem__(self, key, item):
      if key not in self._dictionary:
          raise KeyError(f"The key {key} is not defined.")
      self._dictionary[key] = item
    
    def __getitem__(self, key):
      return self._dictionary[key]
    
    def __repr__(self):
      return self._dictionary.__repr__()
    
    def __str__(self):
      return self._dictionary.__str__()

class Game:
  """
  Game class to be passed throughout the pipeline. Maintains the status of the
  game.
  """
  
  stats = FixedDict(['total','fp_wins','sp_wins','draws'],int)
  
  _diag = iter(([(0,0),(1,1),(2,2)],[(0,2),(1,1),(2,0)]))
  _hori = ([(i,j) for i in range(3)] for j in range(3))
  _vert = ([(j,i) for i in range(3)] for j in range(3))
  combinations = list(itertools.chain(_diag, _hori, _vert))
  
  _sym = {1:'O',-1:'X',0:'_'}
  
  co_ords = [(i,j) for i in range(3) for j in range(3)]
  
  @classmethod
  def get_combinations(cls):
    """Returns all the possible winning combinations"""
    return Game.combinations
  
  def __init__(self, comp_first):
    self.state = FixedDict(((i,j) for i in range(3) for j in range(3)),int)
    self.comp_turn = comp_first
    self.counter = 0
    self.status = "Ongoing"
    
  def __repr__(self):
    return f"Game State:\n{self.get_game_state()}\n{self.get_turn()} to play..."
  
  def __str__(self):
    return self.__repr__()
  
  def __iadd__(self, t):
    """
      Overrides the default __iadd__ so that whenever the state is changed, it 
      updates all the relevant variables. 
    """
    self.counter += 1
    self.state[t] = (-1)**int(self.comp_turn)
    self.comp_turn = not self.comp_turn
    return self
    
  
  def get_game_state(self):
    """Returns a string which contains the game state formatted in a 3x3 grid"""
    return "\n".join("\t".join(Game._sym[self.state[(i,j)]] for i in range(3)) \
                                                            for j in range(3))
  
  def get_turn(self):
    """Returns the name of the current player"""
    return "Player 1" if self.comp_turn else "Player 2"

def main(silent=True, n=1, update_every=100):
  """
  The clear_cell function in google.colab.widgets._grid is buggy. A patch has 
  been sent to their github repo. This is a temperory patch till that patch is 
  made live.
  
  Args:
      silent: If False, shows updates on a game board as they happen. def: True
      n     : Number of games to simulate. def: 1
      update_every: Number of games between stat updates. def: 100
      
  """
  
  main.silent = silent
  if not main.silent:
    main.grid = widgets.Grid(3,3, header_row=False, header_column=False)
    main.eval_grid = widgets.Grid(8,2, header_row=False, header_column=False)
    for i,c in enumerate(Game.get_combinations()):
      main.eval_grid.clear_cell(i,0)
      with main.eval_grid.output_to(i,0):
        print(c)
    
  stats_grid = widgets.Grid(1,1, header_row=False, header_column=False)
  def update_stats():
    stats_grid.clear_cell(0,0)
    with stats_grid.output_to(0,0):
      print(Game.stats)
  
  for i in range(n):
    game = Game(False)
    while evaluate(game):
      show(game)
      move(game)
    if i%update_every==0 and main.silent:
      update_stats()
  update_stats()
    
def evaluate(game):
  """
  Evaluates if the game is finished i.e. if anyone won. Returns a boolean after 
  printing the final state.
  """
  show_eval(game)
  if canPlay(game):
    return True
  else:
    Game.stats['total']+=1
    final_status(game)
    show(game)
    return False
  
def canPlay(game):
  """Checks if the game is over or can be played more."""
  for curr in (sum(game.state[index] for index in c) \
               for c in Game.get_combinations()):
    if curr >= 3:
      game.status = "Player 1 won!"
      Game.stats['fp_wins']+=1
      return False
    elif curr <= -3:
      game.status = "Player 2 won!"
      Game.stats['sp_wins']+=1
      return False
  if game.counter < 9:
    return True
  else:
    game.status = "Draw"
    Game.stats['draws']+=1
    return False

def final_status(game):
  """Prints the final Status of the game if not silent."""
  if main.silent:
    return
  print(game.status)
  
def move(game):
  """
    Depending on player asks first or second player to move. 
  """
  if game.comp_turn:
    comp_move(game)
  else:
    comp_move(game)
  
def human_move(game):
  """
    Takes input for human move.
    //Currently unused.
  """
  game += tuple(map(int,input("Type in co_ordinates i j:").split()))

def comp_move(game,ty='random'):
  """
    Generates a computer move.
    //TODO add uniformly random move function.
    //TODO add min_max function with parameterized depth.
  """
  moves = {'random' : random_move}
  moves[ty](game)

def random_move(game):
  """Generates a (not uniformly) random move."""
  c = choice(Game.co_ords)
  while game.state[c] != 0:
    c = choice(Game.co_ords)
  game += c
  
def show(game):
  """Shows game stat if not silent."""
  if main.silent:
    return
  for i,j in itertools.product(range(3),range(3)):
    main.grid.clear_cell(i,j)
    with main.grid.output_to(i, j):
      print(Game._sym[game.state[(i,j)]])
      
def show_eval(game):
  """Shows game evaluation if not silent."""
  if main.silent:
    return
  for i,c in enumerate(Game.get_combinations()):
    main.eval_grid.clear_cell(i,1)
    with main.eval_grid.output_to(i,1):
      print(sum(game.state[_] for _ in c))
  
if __name__ == '__main__':
  fix_google_colab_widgets()
  main(silent=True, n=10000)

""


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

OrderedDict([('total', 10000), ('fp_wins', 5825), ('sp_wins', 2945), ('draws', 1230)])


<IPython.core.display.Javascript object>